# __Dependencies__

In [ ]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow.python as tf
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.constraints import max_norm
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

# __Marking Keypoints With MP Holistic__

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
#dot, line

In [6]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        #flips frame horizontally --fixes inversed landmarks
        frame = cv2.flip(frame, 1)

        # Check if frame reading was successful
        if not ret or frame is None or frame.shape[0] == 0 or frame.shape[1] == 0:
            print("Error: Failed to capture a valid frame from the webcam.")
            break

        # Print the shape of the frame for further diagnostics
        #print("Frame shape:", frame.shape)
        
        #Make detections (assuming you have the function implemented)
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# __Extracting Keypoints__

In [7]:
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])

In [8]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [9]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#vars for each parts, display 0 if not found

In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# __Creating File Directory__

In [11]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['wave', 'pray', 'iloveyou','miniheart','ok','like','dislike','peace'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [12]:
actions

array(['wave', 'pray', 'iloveyou', 'miniheart', 'ok', 'like', 'dislike',
       'peace'], dtype='<U9')

In [22]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass
#HandRecognition\MP_Data

# __Data Collection__

In [26]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [38]:
cap.release()
cv2.destroyAllWindows()

# __Pre-Procees Data__

In [13]:
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
label_map

{'wave': 0,
 'pray': 1,
 'iloveyou': 2,
 'miniheart': 3,
 'ok': 4,
 'like': 5,
 'dislike': 6,
 'peace': 7}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
np.array(sequences).shape

(240, 30, 1662)

In [17]:
np.array(labels).shape

(240,)

In [18]:
x = np.array(sequences)

In [19]:
x.shape

(240, 30, 1662)

In [20]:
y = to_categorical(labels).astype(int)

In [21]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

# __Build and Train LSTM Neural Network__

In [23]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [29]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(BatchNormalization())
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(BatchNormalization())
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(actions.shape[0], activation='softmax'))

In [30]:
res = [.7, 0.2, 0.1]

In [31]:
actions[np.argmax(res)]

'wave'

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [72]:
def schedule(epoch, lr):
    if epoch < 50:
        return lr
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(schedule)
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback, lr_scheduler])

Epoch 1/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.0954 - categorical_accuracy: 0.9781 - lr: 0.0010
Epoch 2/2000
8/8 [==============================] - 0s 32ms/step - loss: 0.1018 - categorical_accuracy: 0.9737 - lr: 0.0010
Epoch 3/2000
8/8 [==============================] - 0s 32ms/step - loss: 0.1193 - categorical_accuracy: 0.9649 - lr: 0.0010
Epoch 4/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.0903 - categorical_accuracy: 0.9605 - lr: 0.0010
Epoch 5/2000
8/8 [==============================] - 0s 32ms/step - loss: 0.1148 - categorical_accuracy: 0.9518 - lr: 0.0010
Epoch 6/2000
8/8 [==============================] - 0s 32ms/step - loss: 0.0935 - categorical_accuracy: 0.9825 - lr: 0.0010
Epoch 7/2000
8/8 [==============================] - 0s 31ms/step - loss: 0.0657 - categorical_accuracy: 0.9912 - lr: 0.0010
Epoch 8/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.0808 - categorical_accuracy: 0.9825 - lr: 0.0010
Epoch 9/

In [44]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.1985 - categorical_accuracy: 0.9518
Epoch 2/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.2044 - categorical_accuracy: 0.9518
Epoch 3/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.2310 - categorical_accuracy: 0.9430
Epoch 4/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.1968 - categorical_accuracy: 0.9605
Epoch 5/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.2032 - categorical_accuracy: 0.9342
Epoch 6/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.2475 - categorical_accuracy: 0.9123
Epoch 7/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.2564 - categorical_accuracy: 0.9167
Epoch 8/2000
8/8 [==============================] - 0s 32ms/step - loss: 0.2475 - categorical_accuracy: 0.9211
Epoch 9/2000
8/8 [==============================] - 0s 33ms/step - loss: 0.2896 - categorical_accuracy: 0.9254
E

In [73]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            442112    
                                                                 
 batch_normalization_5 (Bat  (None, 30, 64)            256       
 chNormalization)                                                
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 batch_normalization_6 (Bat  (None, 30, 128)           512       
 chNormalization)                                                
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 batch_normalization_7 (Bat  (None, 64)               

In [74]:
model.save('action.keras')

In [75]:
model.load_weights('action.keras')

# __Make Predictions__

In [76]:
res = model.predict(x_test)

1/1 [==============================] - 0s 24ms/step


In [85]:
actions[np.argmax(res[4])]

'miniheart'

In [86]:
actions[np.argmax(y_test[4])]

'miniheart'

In [87]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 22ms/step


In [88]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [89]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[11,  0],
        [ 0,  1]],

       [[10,  0],
        [ 0,  2]],

       [[10,  0],
        [ 0,  2]],

       [[10,  0],
        [ 0,  2]],

       [[ 8,  0],
        [ 0,  4]],

       [[11,  0],
        [ 0,  1]]], dtype=int64)

In [90]:
accuracy_score(ytrue, yhat)

1.0

# __Test in Real Time__